In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',100) # Mudar as configurações originais do PANDAS
pd.set_option('display.max_rows',100) # Mudar as configurações originais do PANDAS

df = pd.read_csv('dados_enem.csv')

df.shape

In [ ]:
df.head()

In [11]:
# Média na nota de Matematica por genero

subset_tp_sexo_m = df.query('TP_SEXO =="M"')

subset_tp_sexo_f = df.query('TP_SEXO =="F"')



In [ ]:
# Nota max genero masculino
subset_tp_sexo_m.NU_NOTA_MT.max()

In [ ]:
# Nota max genero feminino
subset_tp_sexo_f.NU_NOTA_MT.max()

In [ ]:
# Nota max genero feminino
subset_tp_sexo_f.NU_NOTA_MT.mean()

In [ ]:
# Nota max genero masculino
subset_tp_sexo_m.NU_NOTA_MT.mean()

In [19]:
provas = df.columns[(df.columns.str.contains('NOTA')) & (~df.columns.str.contains('COMP'))].tolist()
idCandidato = ['NU_INSCRICAO']

In [ ]:
subset_tp_sexo_m[provas].agg([np.min, np.mean, np.median, np.max]).T

In [ ]:
subset_tp_sexo_f[provas].agg([np.min, np.mean, np.median, np.max]).T

In [ ]:
# Localizar o aluno com nota maxima

id = df.NU_NOTA_MT.idxmax()

df.iloc[id]

In [ ]:
# Localizar o aluno com nota minima

id = df.NU_NOTA_MT.idxmin()

df.iloc[id]

In [ ]:
# Nota minima redacao 
df[df.NU_NOTA_REDACAO != 0].NU_NOTA_REDACAO

# Filtro do que é NAN not a number
df.notna()

In [37]:
mask_1 = df.NU_NOTA_REDACAO.notna()
mask_2 = df.NU_NOTA_REDACAO != 0

subset = df[(mask_1) & (mask_2)]


In [ ]:
subset.NU_NOTA_REDACAO.plot(kind='box')

In [ ]:
subset.NU_NOTA_REDACAO.agg([np.min, np.mean, np.median, np.max])

In [ ]:
# média das colunas
df[provas].mean(axis=0)

In [ ]:
# média das linhas
df[provas].mean(axis=1)

In [ ]:
# Qual a proporção entre os generos ?

df.TP_SEXO.value_counts()
df.groupby(by=['TP_SEXO'])['NU_INSCRICAO'].count()

In [ ]:
# Qual a distribuição dos alunos por tipo de escola?

df.groupby(by=['TP_ESCOLA'])['NU_INSCRICAO'].count().sort_index()

In [ ]:
# Qual o desempenho em matematica por tipo de escola?
# std é o desvio padrão

df.groupby(by=['TP_ESCOLA'])[['NU_NOTA_MT', 'NU_NOTA_CN']].agg([np.min, np.median, np.mean, np.std, np.max]).T

In [ ]:
# Nota media por municipio
df_visao_municipio = (
    df
    .query('NU_NOTA_MT != 0')
    .groupby(by = ['NO_MUNICIPIO_PROVA','CO_MUNICIPIO_PROVA'], as_index=False)['NU_NOTA_MT']
    .agg([np.min, np.mean, np.median, np.max])
    .round(2)
    .rename(columns={'NO_MUNICIPIO_PROVA': 'Municipio',
                     'CO_MUNICIPIO_PROVA': 'COD IBGE',
                     'min': 'Minimo_MT',
                     'mean': 'Media_MT',
                     'median': 'Mediana_MT',
                     'max':'Maximo_MT'})
    .sort_values(by= ['Maximo_MT','Media_MT', 'Mediana_MT'], ascending=False)
    .reset_index(drop=True)
)


In [121]:
df_quantidade_inscritos = (
    df
    .groupby(by = ['NO_MUNICIPIO_PROVA','CO_MUNICIPIO_PROVA'], as_index=False)['NU_INSCRICAO']
    .count()
    .rename(columns={
        'NO_MUNICIPIO_PROVA': 'Municipio',
        'CO_MUNICIPIO_PROVA': 'COD IBGE',
        'NU_INSCRICAO': 'Quantidade_inscritos'
    })
    .sort_values(by=['Quantidade_inscritos'], ascending=False)
)

In [115]:
total = df_quantidade_inscritos.Quantidade_inscritos.sum()

In [116]:
df_quantidade_inscritos['Percentual_inscritos'] = (df_quantidade_inscritos.Quantidade_inscritos / total * 100).round(2)

In [ ]:
# Juntando 2 Data Frames

df_municipio = pd.merge(
    df_visao_municipio.drop([columns = 'Municipio']),
    df_quantidade_inscritos,
    on = 'COD IBGE',
    how ='inner'
)